In [4]:
import os
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DoubleType, TimestampType
from pyspark.sql.functions import col, to_date, concat, lit
os.environ["SPARK_HOME"] = "/home/mate/.local/lib/python3.10/site-packages/pyspark/"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("My Spark Application") \
    .getOrCreate()


24/05/04 21:28:35 WARN Utils: Your hostname, ces-shrd-1 resolves to a loopback address: 127.0.1.1; using 192.168.1.25 instead (on interface wlp0s20f3)
24/05/04 21:28:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/04 21:28:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.sql.functions import col, dayofweek,to_date, month, count, avg
from pyspark.sql import Window
from pyspark.sql.functions import row_number,   sum, when


# Load the CSV file into a DataFrame
csv_file_path_flightdelay = "./full_data_flightdelay.csv"  # Replace with the path to your CSV file


df_flightdelay = spark.read.option("delimiter", ",").option("header", "true").csv(csv_file_path_flightdelay)


# Read the CSV file using the manually defined schema
csv_file_path_weather = "./airport_weather_2019.csv"  # Replace with your file path
df_weather = spark.read.option("delimiter", ",").option("header", "true").csv(csv_file_path_weather)




In [7]:
df_weather.select('DATE').distinct().count()

730

In [8]:
# ----------------------------
# Data cleanup and preparation
# ----------------------------

In [9]:
from pyspark.sql.functions import coalesce

# create new column for month and day_of_week values derived from date
df_day_column = df_weather.withColumn("DATE_NEW", to_date(col("DATE"), "M/d/yyyy"))
df_day_column = df_day_column.withColumn("DATE_NEW", coalesce(df_day_column["DATE_NEW"], to_date(df_day_column["DATE"], 'yyyy-MM-dd')))

df_day_column = df_day_column.withColumn("DAY_OF_WEEK", dayofweek(col("DATE_NEW").alias("DAY_OF_WEEK")))
df_day_column = df_day_column.withColumn("MONTH", month(col("DATE_NEW").alias("MONTH")))
#df_day_column = df_weather.withColumn("DAY_OF_WEEK", dayofweek(col("DATE").alias("DAY_OF_WEEK"))) 




#df_day_column.show(n=2)
df_day_column.createOrReplaceTempView("table1")
df_select = spark.sql("SELECT STATION, NAME,DAY_OF_WEEK,DATE, MONTH, AWND, PRCP, SNOW, SNWD, TAVG, TMAX, TMIN, WDF2 from table1")
#df_select.show(n=5)

grouped_df = df_select.groupBy("MONTH", "NAME").agg(
    avg("AWND").alias("AWND"),
    avg("PRCP").alias("PRCP"),
    avg("SNOW").alias("SNOW"),
    avg("SNWD").alias("SNWD"),
    avg("TAVG").alias("TAVG"),
    avg("TMAX").alias("TMAX"),
    avg("TMIN").alias("TMIN"),
    avg("WDF2").alias("WDF2")
).orderBy("NAME","MONTH")


#grouped_df.show(n=20)

24/05/04 21:28:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
#df_day_column.select('DATE_NEW').distinct().count()

In [11]:
from pyspark.sql.functions import lower, split, col, lit, monotonically_increasing_id


# Normalize joining columns
grouped_df = grouped_df.withColumn("normalized_name", lower(col("name")))
df_flightdelay = df_flightdelay.withColumn("normalized_name", lower(split(col("departing_airport"), " ").getItem(0)))

# Group by to investigate
grouped_df_nn = grouped_df.groupBy("normalized_name").agg(
    count('*').alias('count')
)

grouped_df_name = grouped_df.groupBy("NAME").agg(
    count('*').alias('count')
)

In [12]:
# For 'grouped_df', transforming 'NAME' to lowercase and dropping duplicates based on the 'name' column
grouped_df_lower = grouped_df.select(lower(col("NAME")).alias("name")).dropDuplicates(['name'])

# For 'df_flightdelay', transforming 'DEPARTING_AIRPORT' to lowercase, casting it to string, and dropping duplicates based on the 'departing_airport' column
df_flightdelay_lower = df_flightdelay.select(lower(col("DEPARTING_AIRPORT")).alias("departing_airport")).dropDuplicates(['departing_airport'])



#join providing table that contain in the name column all distinct airports 
#from weather dataset and under departing_flight all distinc airports from delay dataset

#print(grouped_df_lower.count())
#print(df_flightdelay_lower.count())
result_df = df_flightdelay_lower.alias("flight").join(
    grouped_df_lower.alias("grouped"),
    (col("grouped.name").contains(col("flight.departing_airport"))),
    "inner"
).select(
    col("flight.departing_airport").alias("departing_airport"),
    col("grouped.name").alias("name")
)

#result_df.show(n=2)




In [13]:
#modify dataframe such that df_result will contain the airports matched on join 
#and enhanced results will contain the df of unmatched airports for each dataset

# Identifying non-matched entries
non_matched_flight = df_flightdelay_lower.alias("flight").join(
    result_df.alias("result"),
    result_df.departing_airport == df_flightdelay_lower.departing_airport,
    "left_anti"
)

non_matched_grouped = grouped_df_lower.alias("grouped").join(
    result_df.alias("result"),
    result_df.name == grouped_df_lower.name,
    "left_anti"
)


# Add a unique ID to each DataFrame to facilitate the outer join
result_df = result_df.withColumn("id", monotonically_increasing_id())
non_matched_flight = non_matched_flight.withColumn("id", monotonically_increasing_id())
non_matched_grouped = non_matched_grouped.withColumn("id", monotonically_increasing_id())


# Perform the outer joins using the unique IDs, result_df is now composed of matched airports
enhanced_result_df = result_df.join(non_matched_flight, "id", "outer" ).join(non_matched_grouped, "id", "outer" )
enhanced_result_df = enhanced_result_df.drop("id")

# Show the enhanced DataFrame with additional columns
#enhanced_result_df.printSchema()

# Select columns, get rid of duplicates
selected_columns = [col for col in enhanced_result_df.columns if col != 'name' and col != 'departing_airport'] + ['grouped.name'] + ['flight.departing_airport']

#will contain unmatched airports for each dataset
enhanced_result_df = enhanced_result_df.select(selected_columns)
enhanced_result_df.drop('name','departing_airport')
#enhanced_result_df.show(n=2)


DataFrame[]

In [14]:
# create dataframe that contains airports matched and unmatched result from the join
# Rename columns in result_df
result_df = result_df.withColumnRenamed("name", "weather_matched") \
                     .withColumnRenamed("departing_airport", "delay_matched")

# Rename columns in enhanced_result_df
enhanced_result_df = enhanced_result_df.withColumnRenamed("name", "weather_unmatched") \
                                       .withColumnRenamed("departing_airport", "delay_unmatched")

# Optional: If you need to ensure the rows are matched by order, add an index column to each DataFrame
result_df = result_df.withColumn("index", monotonically_increasing_id())
enhanced_result_df = enhanced_result_df.withColumn("index", monotonically_increasing_id())

# Join DataFrames on the index column
matched_and_unmatched_airports = result_df.join(
    enhanced_result_df,
    on="index",
    how="outer"  # Use "outer" to include all rows from both DataFrames
)

# Drop the index column as it's no longer needed after joining
matched_and_unmatched_airports = matched_and_unmatched_airports.drop("index", 'name', 'departing_airport')

# Show the resulting DataFrame structure
#matched_and_unmatched_airports.show(n = 76)

In [15]:
#count the amount of airports for each clumn in the enhanced dataset dataframe 

non_null_name_count = result_df.filter(col("name").isNotNull()).count()
non_null_name_count1 = result_df.filter(col("departing_airport").isNotNull()).count()
non_null_name_count2 = enhanced_result_df.filter(col("weather_unmatched").isNotNull()).count()
non_null_name_count3 = enhanced_result_df.filter(col("delay_unmatched").isNotNull()).count()
#print("Number of non-null strings in the 'name' column:", non_null_name_count, non_null_name_count1,non_null_name_count2, non_null_name_count3)

# Display the filtered DataFrame and print the counts



In [16]:
import pandas as pd

# Initialize a list to store the parsed data
data = []

# Open the text file and parse it line by line
with open('./airports.txt', 'r') as file:
    for line in file:
        # Split the line by comma to extract the needed parts
        parts = line.split(',')
        
        # Check if the line has enough parts to avoid index errors
        if len(parts) >= 4:
            # Extract and clean the desired parts
            # Remove quotation marks and extra spaces if present
            name = parts[1].strip('"').strip()
            city = parts[2].strip('"').strip()
            country = parts[3].strip('"').strip()
            
            # Combine the first two parts into one column, and keep the country as the second column
            combined = f"{name}, {city}, {country}"
            data.append(combined)

# Create a DataFrame from the list
df_airports = pd.DataFrame(data, columns=['Airport and City'])


In [17]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [18]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Get all airport names into one table
df_delay_unique_airport = df_flightdelay.select('DEPARTING_AIRPORT').distinct().withColumnRenamed("DEPARTING_AIRPORT", "airport")
df_weather_unique_airport = df_weather.select('NAME').distinct().withColumnRenamed("NAME", "airport")

df_union = df_delay_unique_airport.union(df_weather_unique_airport)

def filter_out_useless_parts_of_string(airport_name):
    useless_words = ["international", "airport", "regional"]

    modified = airport_name.lower()
    for word in useless_words:
        modified = modified.replace(word,"")
        
    return modified
    
filter_string_udf = udf(filter_out_useless_parts_of_string, StringType())

df_union = df_union.select(filter_string_udf(col('airport'))).withColumnRenamed("filter_out_useless_parts_of_string(airport)", "airport")
#df_union.show(n=5)



In [19]:
from fuzzywuzzy import process, fuzz

def get_matches(df1, col1, df2, col2, threshold=40):
    # Convert each column to a list for processing, ensuring to drop NA values
    list1 = df1[col1].dropna().tolist()
    list2 = df2[col2].dropna().tolist()

    # Find best matches with a score above the threshold
    matches = []
    for item in list1:
        # Use process.extractOne to find the best match for each item from list1 in list2
        best_match = process.extractOne(item, list2, scorer=fuzz.token_set_ratio)
        if best_match and best_match[1] >= threshold:
            matches.append((item, best_match[0], best_match[1]))

    # Return matches as a DataFrame for better visualization
    return pd.DataFrame(matches, columns=[col1, col2 + '_match', 'Score'])

In [20]:
# Add official airports using fuzzywuzzy
# Assuming 'matched_and_unmatched_airports' is your PySpark DataFrame
pandas_df = df_union.toPandas()  # Convert to Pandas DataFrame

# Example usage (ensure df1 and df2 are already defined and loaded with your data)
df_matches_airports = get_matches(pandas_df, 'airport', df_airports, 'Airport and City')

spark_df_airports = spark.createDataFrame(df_matches_airports)
# Show the DataFrame to verify conversion


In [69]:
joining_table = spark_df_airports.select("airport", "Airport and City_match")
#joining_table.show(n=2)
delay_table = df_flightdelay.withColumn("DEPARTING_AIRPORT", filter_string_udf('DEPARTING_AIRPORT'))
#delay_table.show(n=2)
weather_table = df_day_column.withColumn("NAME", filter_string_udf('NAME'))
#weather_table.show(n=2)

#previous_arprt_table = df_flightdelay.withColumn("PREVIOUS_AIRPORT", filter_string_udf('PREVIOUS_AIRPORT'))
#previous_arprt_joined = previous_arprt_table.join(joining_table, joining_table.airport == previous_arprt_table.PREVIOUS_AIRPORT, 'inner')

#delay_joined = delay_table.join(joining_table, joining_table.airport == delay_table.DEPARTING_AIRPORT, 'inner')
#weather_joined = weather_table.join(joining_table, joining_table.airport == weather_table.NAME, 'inner')

In [22]:



#delay_joined.show(n=3)
#weather_joined.show(n=3)

# Convert integer columns in df1 to strings
weather_joined = weather_joined.withColumn("MONTH", col("MONTH").cast("string")) \
         .withColumn("DAY_OF_WEEK", col("DAY_OF_WEEK").cast("string"))

result_joined = delay_joined.join(weather_joined, ["MONTH","DAY_OF_WEEK","Airport and City_match"], 'inner')

#result_joined.show(n=2)

#weather_join_delay = weather_joined.join(delay_joined, )

In [23]:
#result_joined.count()

In [24]:
#result_joined.printSchema()

In [25]:
# Drop unnecessary columns
# LATITUDE, LONGITUDE, STATION, MONTH, airport, normalized_name, NAME, _c0
result_joined = result_joined.drop("LATITUDE", "LONGITUDE", "STATION", "MONTH", \
                                   "airport", "normalized_name", "NAME", "_c0", \
                                   "DATE", "AIRLINE_FLIGHTS_MONTH", "AVG_MONTHLY_PASS_AIRLINE" \
                                   "DEPARTING_AIRPORT", "PGTM", "WDF5", "WDF2", "WSF2", "WSF5", \
                                   "SN32", "SX32", "TOBS","WESD", "PSUN","TSUN")

df = result_joined
#result_joined.count()
#result_joined.select("CONCURRENT_FLIGHTS").filter(col("CONCURRENT_FLIGHTS").isNotNull()).count()

In [26]:
df = df.withColumn("TMIN", df["TMIN"].cast("float"))
df = df.withColumn("PRCP", df["PRCP"].cast("float"))


#print(df.count())


#count = df.filter(((col("PRCP") > 0.0) & (col("TMIN") < 3.0)) & (col("SNOW").isNull() | (col("SNOW") == ''))).count()

df = df.withColumn("SNOW", when(col("TMIN") > 3, 0).otherwise(col("SNOW")))
df = df.withColumn("SNOW", when( \
                    (col("SNOW").isNull() | (col("SNOW") == '')) & (col("PRCP") > 0), col("PRCP") \
                               ).otherwise(lit(0)))


In [27]:
# Simplify temperatures, create new flag column EXTREME_WEATHER based on TMIN and TMAX and drop all others


# Since TMAX and TMIN are strings, you need to convert them to integers before comparison
df = df.withColumn("TMAX", df["TMAX"].cast("integer"))
df = df.withColumn("TMIN", df["TMIN"].cast("integer"))

# Creating the EXTREME_WEATHER column based on the conditions provided
df = df.withColumn("EXTREME_WEATHER", 
                   when((col("TMAX") > 40) | (col("TMIN") < 0), 1)
                   .otherwise(0))

df = df.drop("TMIN", "TMAX", "TAVG")

In [28]:
from pyspark.sql.functions import expr

# Replace all WT** with column which adds these extreme weather conditions into one value
values_as_strings = [f"WT{i:02}" for i in range(1, 12)]
#print(values_as_strings)
#print("+".join(values_as_strings))

for column_name in values_as_strings:
    df = df.withColumn(column_name, df[column_name].cast("integer"))

In [29]:
#df.printSchema()


In [31]:
from functools import reduce

total_wt_column = reduce(lambda a, b: a + b, [coalesce(col(c), lit(0)) for c in values_as_strings])

df = df.withColumn('EXTREME_WEATHER_WT', total_wt_column)

df = df.drop('WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09', 'WT10', 'WT11')

#df.printSchema()

#df.groupBy("EXTREME_WEATHER_WT").count().orderBy("EXTREME_WEATHER_WT").show()
#df.select("EXTREME_WEATHER_WT").show(n=50)

In [46]:
df.printSchema()

root
 |-- CARRIER_NAME: string (nullable = true)
 |-- Airport and City_match: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- SEGMENT_NUMBER: string (nullable = true)
 |-- CONCURRENT_FLIGHTS: string (nullable = true)
 |-- NUMBER_OF_SEATS: string (nullable = true)
 |-- AIRPORT_FLIGHTS_MONTH: string (nullable = true)
 |-- AIRLINE_AIRPORT_FLIGHTS_MONTH: string (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRPORT: string (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRLINE: string (nullable = true)
 |-- GROUND_SERV_PER_PASS: string (nullable = true)
 |-- PLANE_AGE: string (nullable = true)
 |-- DEPARTING_AIRPORT: string (nullable = true)
 |-- PREVIOUS_AIRPORT: string (nullable = true)
 |-- SNOW: float (nullable = true)
 |-- SNWD: string (nullable = true)
 |-- DATE_NEW: date (nullable = true)
 |-- EXTREME_WEATHER: integer (nullable = false)
 |-- EXTREME_WEATHER_WT: integer (nullable = false)


In [33]:
from pyspark.sql.functions import col, when, count

# Define a function to count nulls and empty strings
def count_nulls_and_empties(df):
    # Use aggregation to sum up each condition of being null or empty across all columns
    exprs = [count(when(col(c).isNull() | (col(c) == ""), c)).alias(c) for c in df.columns]
    return df.agg(*exprs)

# Apply the function to count nulls and empties
#nulls_and_empties_count = count_nulls_and_empties(df)

# Show the result
#nulls_and_empties_count.show()

#df.count()

In [47]:
# Fill in missing values, for example AWND and PRCP

from pyspark.sql.functions import avg, col, coalesce, month, median
from pyspark.sql.window import Window

# Define a window spec partitioned by month
window_spec = Window.partitionBy(month("DATE_NEW"))

# Assuming 'column_name' is the column with null values you want to fill
avg_column = avg(col("AWND")).over(window_spec)
avg_prcp = avg(col("PRCP")).over(window_spec)
med_column = median(col("FLT_ATTENDANTS_PER_PASS")).over(window_spec)


# Replace nulls with the average of that month
df = df.withColumn("AWND_filled", coalesce(col("AWND"), avg_column))
df = df.withColumn("PRCP_filled", coalesce(col("PRCP"), avg_prcp))
df = df.withColumn("DISTANCE_GROUP_filled", coalesce(col("DISTANCE_GROUP"), lit("1")))
df = df.withColumn("FLT_ATTENDANTS_PER_PASS_filled", coalesce(col("FLT_ATTENDANTS_PER_PASS"), med_column))


df = df.drop("AWND").withColumnRenamed("AWND_filled", "AWND")
df = df.drop("PRCP").withColumnRenamed("PRCP_filled", "PRCP")
df = df.drop("DISTANCE_GROUP").withColumnRenamed("DISTANCE_GROUP_filled", "DISTANCE_GROUP")
df = df.drop("FLT_ATTENDANTS_PER_PASS").withColumnRenamed("FLT_ATTENDANTS_PER_PASS_filled", "FLT_ATTENDANTS_PER_PASS")

#nulls_and_empties_count = count_nulls_and_empties(df)

# Show the result
#nulls_and_empties_count.show()


In [48]:
from pyspark.sql.functions import round

calculate_flights = df.select("CARRIER_NAME", "Airport and City_match", "DATE_NEW")

calculate_flights = calculate_flights.withColumn("MONTH", month(col("DATE_NEW").alias("MONTH")))

count = calculate_flights.groupBy("CARRIER_NAME", "Airport and City_match", "MONTH").count()
#print(count.show())

count = count.groupBy("CARRIER_NAME", "Airport and City_match") \
                   .agg(round(avg("count")).alias("monthly_avg_count"))
#print(count.show())

In [49]:
rsdf = df.join(
    count,
    ["CARRIER_NAME", "Airport and City_match"],
    "inner"
)

rsdf = rsdf.withColumn("AIRLINE_AIRPORT_FLIGHTS_MONTH",\
                       when(\
                           (col("AIRLINE_AIRPORT_FLIGHTS_MONTH").isNull() | (col("AIRLINE_AIRPORT_FLIGHTS_MONTH") == ''))
                           , col("monthly_avg_count")).otherwise(col("AIRLINE_AIRPORT_FLIGHTS_MONTH")))

rsdf = rsdf.drop("monthly_avg_count")

#rsdf.show()

result_df = result_joined


In [50]:
#check if there are any null values left
df = rsdf

#nulls_and_empties_count = count_nulls_and_empties(df)

# Show the result
#nulls_and_empties_count.show()
#df.printSchema()
#df.show(n=2)

In [51]:
# ------------------------------------
# Converting from numerical to nominal
# ------------------------------------

In [52]:
# Convert precipitation from numerical to nominal

# Calculate the quantile thresholds
#thresholds = result_df.approxQuantile("PRCP", [0.33, 0.67], 0.01)  # 0.01 is the relative error

# Categorize based on quantile thresholds
#result_df = result_df.withColumn(
#    "precip_category",
#    when(col("PRCP") <= thresholds[0], "low")
#    .when(col("PRCP") <= thresholds[1], "medium")
#    .otherwise("high")
#)

# Show the resulting DataFrame
#result_df.select("PRCP", "precip_category").show()

In [53]:
# Transform weekday from numerical to nominal

# Weekday mapping dictionary
month_dict = {
    '1': 'Monday', '2': 'Tuesday', '3': 'Wednesday', '4': 'Thursday', 
    '5': 'Friday', '6': 'Saturday', '7': 'Sunday'}

# Define the UDF to convert numerical months to names
def convert_weekday_to_name(weekday):
    return month_dict.get(str(weekday), "Unknown")

convert_weekday_udf = udf(convert_weekday_to_name, StringType())

# Apply the UDF to create a new column with month names
df_with_months = result_df.withColumn("DAY_OF_WEEK_NAME", convert_weekday_udf(result_df["DAY_OF_WEEK"]))
#df_with_months.show(n=1)
df_with_months = df_with_months.drop("DAY_OF_WEEK").withColumnRenamed("DAY_OF_WEEK_NAME", "DAY_OF_WEEK")

In [54]:
# NUMBER_OF_SEATS into nominal

# Categorize based on research
result_df = result_df.withColumn(
    "NUMBER_OF_SEATS_NOM",
    when(col("NUMBER_OF_SEATS") <= 100, "Small")
    .when(col("NUMBER_OF_SEATS") <= 200, "Medium")
    .when(col("NUMBER_OF_SEATS") <= 400, "Large")
    .otherwise("Jumbo")
)

# Replace NUMBER_OF_SEATS column with the nominal one
result_df = result_df.drop("NUMBER_OF_SEATS").withColumnRenamed("NUMBER_OF_SEATS_NOM", "NUMBER_OF_SEATS")

In [55]:
# Plane age into nominal

# Categorize based on research
result_df = result_df.withColumn(
    "PLANE_AGE_NOM",
    when(col("PLANE_AGE") <= 10, "New")
    .when(col("PLANE_AGE") <= 20, "Standard")
    .otherwise("Old")
)

# Replace PLANE_AGE column with the nominal one
result_df = result_df.drop("PLANE_AGE").withColumnRenamed("PLANE_AGE_NOM", "PLANE_AGE")


In [56]:
result_df.printSchema()

root
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- Airport and City_match: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- DISTANCE_GROUP: string (nullable = true)
 |-- SEGMENT_NUMBER: string (nullable = true)
 |-- CONCURRENT_FLIGHTS: string (nullable = true)
 |-- CARRIER_NAME: string (nullable = true)
 |-- AIRPORT_FLIGHTS_MONTH: string (nullable = true)
 |-- AIRLINE_AIRPORT_FLIGHTS_MONTH: string (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRPORT: string (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRLINE: string (nullable = true)
 |-- FLT_ATTENDANTS_PER_PASS: string (nullable = true)
 |-- GROUND_SERV_PER_PASS: string (nullable = true)
 |-- DEPARTING_AIRPORT: string (nullable = true)
 |-- PREVIOUS_AIRPORT: string (nullable = true)
 |-- AWND: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- SNOW: string (nullable = true)
 |-- SNWD: string (nullable = true)
 |-- TAVG: string (nullable = true)
 |-- TMAX: string

In [57]:
# ------------------
# Data analysis
# ------------------

In [ ]:
# Split data into training and testing batches
fractions = {label: 0.1 for label in df.select("DEP_DEL15").distinct().rdd.flatMap(lambda x: x).collect()}
sampled_df = joined_df.stat.sampleBy("DEP_DEL15", fractions, seed=1234)

# Show the sampled data distribution
sampled_df.groupBy("DEP_DEL15").count().show()

# Split the DataFrame into training (60%) and test (40%) sets
train_df, test_df = sampled_df.randomSplit([0.6, 0.4], seed=1234)

# Show the size of each set
print("Training Dataset Count: " + str(train_df.count()))
print("Testing Dataset Count: " + str(test_df.count()))

In [ ]:
# Calculate staistics for numerical attributes
numeric_columns = [
    "DAY_OF_WEEK", "DEP_DEL15", "SEGMENT_NUMBER", "CONCURRENT_FLIGHTS",
    "NUMBER_OF_SEATS", "AIRPORT_FLIGHTS_MONTH",
    "AIRLINE_AIRPORT_FLIGHTS_MONTH", "AVG_MONTHLY_PASS_AIRPORT",
    "GROUND_SERV_PER_PASS", "SNOW", "EXTREME_WEATHER", "EXTREME_WEATHER_WT",
    "AWND", "PRCP", "DISTANCE_GROUP", "FLT_ATTENDANTS_PER_PASS"
]

# Cast each numeric column to float
for column in numeric_columns:
    df = df.withColumn(column, col(column).cast('integer'))
    
stats_df = df.describe()
stats_df.show()

In [60]:
# Create a 10% sample

# Check distinct values in the DEP_DEL15 column and their distribution if needed
snow_distribution = df.groupBy("DEP_DEL15").count().collect()

# Create a dictionary for fractions
fractions = {row['DEP_DEL15']: 0.1 for row in snow_distribution}  # 10% sample for each category in SNOW

# Sample the data maintaining the distribution of the SNOW column
sampled_df = df.stat.sampleBy("DEP_DEL15", fractions, seed=42)

# Check the distribution in the sampled DataFrame
sampled_distribution = sampled_df.groupBy("DEP_DEL15").count().show()

# Optionally, compare it to the original distribution
print("Original Distribution:")
for row in snow_distribution:
    print("SNOW: {}, Count: {}".format(row['DEP_DEL15'], row['count']))

print("\nSampled Distribution:")
sampled_distribution


24/05/04 21:37:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:37:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:37:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:37:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:37:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:37:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:37:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:38:00 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 21:38:27 WARN RowBasedKeyValueBatch: Calling spill() on

+---------+-------+
|DEP_DEL15|  count|
+---------+-------+
|        0|1228130|
|        1| 278647|
+---------+-------+

Original Distribution:
SNOW: 0, Count: 12277239
SNOW: 1, Count: 2774410

Sampled Distribution:


In [61]:
#10% of dataset
df = sampled_df

In [ ]:
!pip install seaborn
import matplotlib.pyplot as plt
import seaborn as sns  # For better visual aesthetics

# Setting a style
sns.set(style="whitegrid")

# List of nominal columns
nominal_columns = [
    "CARRIER_NAME",
    "Airport and City_match",
    "DEP_TIME_BLK",
    "DEPARTING_AIRPORT",
    "PREVIOUS_AIRPORT",
    "DATE_NEW",
    "PLANE_AGE"
]

# Create a figure to hold the subplots
fig, axes = plt.subplots(nrows=len(nominal_columns), figsize=(12, 6 * len(nominal_columns)))  # Adjusted size for better fit

# Loop through each nominal column and create a histogram
for i, column in enumerate(nominal_columns):
    # Count the occurrences of each category in the column
    category_counts = df.groupBy(column).count().toPandas().sort_values(by='count', ascending=False)

    # If there are many categories, you might want to limit the number of categories displayed
    if len(category_counts) > 20:  # Adjust the threshold as needed
        category_counts = category_counts.head(20)  # Only show top 20 categories
    
    # Plotting the histogram for the column
    sns.barplot(x=column, y='count', data=category_counts, ax=axes[i], palette='viridis')  # Using seaborn for a better-looking plot
    axes[i].set_title(f'Histogram of {column}')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Counts')
    axes[i].tick_params(axis='x', rotation=45)  # Rotate labels to prevent overlap

# Adjust layout to prevent overlap of subplots
plt.tight_layout()
plt.show()


In [ ]:
# calculate :  pomerového kritéria – informačného zisku voči cieľovému atribútu (klasifikačná úloha), pre nominálne atribúty 
from pyspark.sql.functions import col, log2, sum as sqlsum

def entropy(df, column):
    """Calculate the entropy of a given DataFrame column."""
    # Calculate the total number of records in the DataFrame
    total = df.count()
    # Calculate the probabilities for each distinct value in the column
    probabilities = df.groupBy(column).count().select((col('count') / total).alias('probability'))
    # Calculate the entropy using the probability data
    entropy = probabilities.withColumn('entropy_part', - col('probability') * log2(col('probability'))).agg(sqlsum('entropy_part')).collect()[0][0]
    return entropy if entropy is not None else 0

def information_gain(df, attribute, target):
    """Calculate the information gain of a column with respect to the target."""
    total_entropy = entropy(df, target)
    total_count = df.count()

    # Get distinct values of the attribute
    attribute_values = df.select(attribute).distinct().collect()

    conditional_entropy = 0
    for row in attribute_values:
        attr_val = row[attribute]
        # Filter the DataFrame by each attribute value and calculate its entropy
        df_subset = df.filter(col(attribute) == attr_val)
        subset_count = df_subset.count()
        if subset_count > 0:
            # Compute entropy for each subset
            subset_entropy = entropy(df_subset, target)
            conditional_entropy += (subset_count / total_count) * subset_entropy

    # Information gain is the total entropy minus the conditional entropy
    return total_entropy - conditional_entropy

# Example usage
nominal_columns = ["CARRIER_NAME", "Airport and City_match", "DEP_TIME_BLK", "DEPARTING_AIRPORT", "PREVIOUS_AIRPORT", "DATE_NEW"]

information_gains = {}
for column in nominal_columns:
    ig = information_gain(df, column, 'DEP_DEL15')  # Assuming DEP_DEL15 is your target column
    information_gains[column] = ig
    print(f'Information Gain for {column}: {ig}')


In [62]:
df.printSchema()

root
 |-- CARRIER_NAME: string (nullable = true)
 |-- Airport and City_match: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- DEP_DEL15: string (nullable = true)
 |-- DEP_TIME_BLK: string (nullable = true)
 |-- SEGMENT_NUMBER: string (nullable = true)
 |-- CONCURRENT_FLIGHTS: string (nullable = true)
 |-- NUMBER_OF_SEATS: string (nullable = true)
 |-- AIRPORT_FLIGHTS_MONTH: string (nullable = true)
 |-- AIRLINE_AIRPORT_FLIGHTS_MONTH: string (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRPORT: string (nullable = true)
 |-- AVG_MONTHLY_PASS_AIRLINE: string (nullable = true)
 |-- GROUND_SERV_PER_PASS: string (nullable = true)
 |-- PLANE_AGE: string (nullable = true)
 |-- DEPARTING_AIRPORT: string (nullable = true)
 |-- PREVIOUS_AIRPORT: string (nullable = true)
 |-- SNOW: float (nullable = true)
 |-- SNWD: string (nullable = true)
 |-- DATE_NEW: date (nullable = true)
 |-- EXTREME_WEATHER: integer (nullable = false)
 |-- EXTREME_WEATHER_WT: integer (nullable = false)


In [ ]:
# ---------------------------
# Model creation and training
# ---------------------------

In [ ]:
df = df.drop("DEPARTING_AIRPORT")

In [ ]:
# Plane age into numerical

# Categorize based on research
df = df.withColumn(
    "PLANE_AGE_NOM",
    when(col("PLANE_AGE") == "New", 1)
    .when(col("PLANE_AGE") == "Standard", 2)
    .otherwise(3)
)

# Replace PLANE_AGE column with the nominal one
df = df.drop("PLANE_AGE").withColumnRenamed("PLANE_AGE_NOM", "PLANE_AGE")

24/05/04 22:28:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:28:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:34:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:34:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:39:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:39:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:39:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:39:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/04 22:39:44 WARN RowBasedKeyValueBatch: Calling spill() on

In [ ]:
# NUMBER_OF_SEATS into numerical category

# Categorize based on research
result_df = result_df.withColumn(
    "NUMBER_OF_SEATS_NOM",
    when(col("NUMBER_OF_SEATS") == "Small", 1)
    .when(col("NUMBER_OF_SEATS") == "Medium", 2)
    .when(col("NUMBER_OF_SEATS") == "Large", 3)
    .otherwise(4)
)

# Replace NUMBER_OF_SEATS column with the nominal one
result_df = result_df.drop("NUMBER_OF_SEATS").withColumnRenamed("NUMBER_OF_SEATS_NOM", "NUMBER_OF_SEATS")

In [ ]:
# DAY_OF_WEEK into numerical form
df = df.withColumn("day_cos", cos(radians(col("DAY_OF_WEEK") * (360/7))))
df = df.withColumn("day_sin", sin(radians(col("DAY_OF_WEEK") * (360/7))))

In [ ]:
df.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import col, sqrt, pow, mean, stddev


# Convert necessary columns from string to appropriate numeric types
columns_to_cast = ["SEGMENT_NUMBER", "CONCURRENT_FLIGHTS", "NUMBER_OF_SEATS", 
                   "AIRPORT_FLIGHTS_MONTH", "AIRLINE_AIRPORT_FLIGHTS_MONTH", 
                   "AVG_MONTHLY_PASS_AIRPORT", "GROUND_SERV_PER_PASS", 
                   "SNOW", "AWND", "PRCP", "DISTANCE_GROUP", "FLT_ATTENDANTS_PER_PASS"]

for col_name in columns_to_cast:
    df = df.withColumn(col_name, df[col_name].cast('float'))

# Handle missing values if necessary, here using median or a predefined constant
# Assuming missing values handled outside of this snippet or need to be addressed
# df = df.fillna({'someColumn': value})

# Assemble features, excluding the target 'DEP_DEL15'
assembler = VectorAssembler(
    inputCols=["SEGMENT_NUMBER", "CONCURRENT_FLIGHTS", "NUMBER_OF_SEATS", "AIRPORT_FLIGHTS_MONTH",
               "AIRLINE_AIRPORT_FLIGHTS_MONTH", "AVG_MONTHLY_PASS_AIRPORT", "GROUND_SERV_PER_PASS",
               "SNOW", "AWND", "PRCP", "DISTANCE_GROUP", "FLT_ATTENDANTS_PER_PASS"],
    outputCol="features"
)
featured_df = assembler.transform(df)

# Scale features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scaled_df = scaler.fit(featured_df).transform(featured_df)

# Determine the optimal number of clusters using the Silhouette Score
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='scaledFeatures', metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette_scores = []
for k in range(2, 10):  # Adjust the range of k as needed
    kmeans = KMeans(featuresCol="scaledFeatures", k=k)
    model = kmeans.fit(scaled_df)
    predictions = model.transform(scaled_df)
    score = evaluator.evaluate(predictions)
    silhouette_scores.append((k, score))

# Select the best k with the highest silhouette score
best_k = max(silhouette_scores, key=lambda item: item[1])[0]

# Build the final KMeans model with the best k
final_kmeans = KMeans(featuresCol="scaledFeatures", k=best_k)
final_model = final_kmeans.fit(scaled_df)
final_predictions = final_model.transform(scaled_df)

# Calculate distances to cluster center for anomaly detection
centers = final_model.clusterCenters()
get_distance = lambda features, center: sqrt(sum([pow((f - c), 2) for f, c in zip(features, center)]))
udf_get_distance = F.udf(get_distance, FloatType())

# Adding a column for distances to each center
for i, center in enumerate(centers):
    final_predictions = final_predictions.withColumn(f"dist_to_center_{i}", udf_get_distance(col("scaledFeatures"), F.array([F.lit(x) for x in center])))

# Setting anomaly detection threshold
distance_desc = final_predictions.describe(["dist_to_center_0"])  # Assuming using distance to center of first cluster
mean_distance = float(distance_desc.filter("summary = 'mean'").select("dist_to_center_0").collect()[0][0])
stddev_distance = float(distance_desc.filter("summary = 'stddev'").select("dist_to_center_0").collect()[0][0])
threshold = mean_distance + 3 * stddev_distance
anomalies = final_predictions.filter(final_predictions["dist_to_center_0"] > threshold)

# Show results
anomalies.show()

24/05/05 16:31:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:31:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:50:05 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:50:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:50:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:56:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:56:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:56:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 16:56:29 WARN RowBasedKeyValueBatch: Calling spill() on

In [74]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# Indexing and Encoding for 'CARRIER_NAME'
carrier_name_indexer = StringIndexer(inputCol="CARRIER_NAME", outputCol="CARRIER_NAME_Index")
carrier_name_encoder = OneHotEncoder(inputCol="CARRIER_NAME_Index", outputCol="CARRIER_NAME_Encoded")

# Indexing and Encoding for 'PREVIOUS_AIRPORT'
prev_airport_indexer = StringIndexer(inputCol="PREVIOUS_AIRPORT", outputCol="PREVIOUS_AIRPORT_Index")
prev_airport_encoder = OneHotEncoder(inputCol="PREVIOUS_AIRPORT_Index", outputCol="PREVIOUS_AIRPORT_Encoded")

# Indexing and Encoding for 'Airport and City_match'
city_match_indexer = StringIndexer(inputCol="Airport and City_match", outputCol="Airport_and_City_match_Index")
city_match_encoder = OneHotEncoder(inputCol="Airport_and_City_match_Index", outputCol="Airport_and_City_match_Encoded")

# Assemble features into a single vector
assembler = VectorAssembler(
    inputCols=["PREVIOUS_AIRPORT_Encoded", "Airport_and_City_match_Encoded", "CARRIER_NAME_Encoded"] + ["SNOW"],  # Replace "other_features" with actual feature names
    outputCol="features"
)

# Pipeline all preprocessing steps
pipeline = Pipeline(stages=[
    prev_airport_indexer, prev_airport_encoder, city_match_indexer, city_match_encoder, carrier_name_indexer, carrier_name_encoder, assembler
])

# Fit and Transform
model = pipeline.fit(df)

# Unclear if works correctly
#transformed_df = model.transform(df)
print(model)

PipelineModel_63b22420a9e9


In [ ]:
### AT THIS POINT DEFINE train_data AND test_data
# Like so for example: train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=42)

In [76]:
transformed_df = model.transform(df)
transformed_df.show(n=2)

+--------------------+----------------------+-----------+---------+------------+--------------+------------------+---------------+---------------------+-----------------------------+------------------------+------------------------+--------------------+---------+-----------------+--------------------+----+----+----------+---------------+------------------+-----+----+--------------+-----------------------+----------------------+------------------------+----------------------------+------------------------------+------------------+--------------------+--------------------+
|        CARRIER_NAME|Airport and City_match|DAY_OF_WEEK|DEP_DEL15|DEP_TIME_BLK|SEGMENT_NUMBER|CONCURRENT_FLIGHTS|NUMBER_OF_SEATS|AIRPORT_FLIGHTS_MONTH|AIRLINE_AIRPORT_FLIGHTS_MONTH|AVG_MONTHLY_PASS_AIRPORT|AVG_MONTHLY_PASS_AIRLINE|GROUND_SERV_PER_PASS|PLANE_AGE|DEPARTING_AIRPORT|    PREVIOUS_AIRPORT|SNOW|SNWD|  DATE_NEW|EXTREME_WEATHER|EXTREME_WEATHER_WT| AWND|PRCP|DISTANCE_GROUP|FLT_ATTENDANTS_PER_PASS|PREVIOUS_AIRPO

24/05/05 00:06:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/05 00:06:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [ ]:
# Naive Bayes model
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler


# Indexing string columns to convert to numeric values for model input
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_index").fit(df)
    for column in list(set(df.columns) - set(['DEP_DEL15'])) # Assuming 'DEP_DEL15' is the label
]

# Use Pipeline to apply indexers
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(df).transform(df)

# VectorAssembler to create feature vector
assembler = VectorAssembler(
    inputCols=[column + "_index" for column in list(set(df.columns) - set(['DEP_DEL15']))],
    outputCol="features"
)

data = assembler.transform(df_r)

# Select features and label for model training
final_data = data.select(col("features"), col("DEP_DEL15").alias("label"))

from pyspark.ml.classification import NaiveBayes

# Initialize the Naive Bayes model
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# Train the model
model = nb.fit(train_data)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

In [ ]:
# Support Vector Machine
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Initialize the LinearSVC model

svm = LinearSVC(featuresCol="features", labelCol="label", maxIter=100, regParam=0.1)

# Fit the model on training data
svm_model = svm.fit(train_data)

# Predictions
predictions = svm_model.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Area under ROC: {auc:.2f}")

In [ ]:
# Random forest
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=10)

# Train the model
rf_model = rf.fit(train_data)

# Make predictions
predictions = rf_model.transform(test_data)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % accuracy)